##### Copyright 2018 The TensorFlow Constrained Optimization Authors. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the \"License\"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

> http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

## Problem Setup

This is a simple example of recall-constrained optimization on simulated data: we seek a classifier that minimizes the average hinge loss while constraining recall to be at least 90%.

We'll start with the required imports&mdash;notice the definition of `tfco`:

In [1]:
import math
import numpy as np
from six.moves import xrange
import tensorflow as tf
import pandas as pd
from sklearn import model_selection
from sklearn import metrics

We load the [https://www.researchconnections.org/icpsrweb/instructors/studies/36151] and do some pre-processing. The dataset is based on India's Human Development Survey, is to predict whether someone who has higher education is appointed as a teaching staff or not. We construct three protected groups, two based on gender (SS7 - Male and Female) and two based on religion (SS11- Hindu, Muslim, Christian, Sikh, Tribal, etc) and on Jati Code (SS12 - Brahmi, Forward Class or Backward Class OBSC, etc).

In [2]:
# Use the GitHub version of TFCO
#!pip install git+https://github.com/google-research/tensorflow_constrained_optimization.git
import tensorflow_constrained_optimization as tfco

In [2]:
dataset_path = "data/staffing-teaching/School-Staff-Data.csv"

# Read dataset from the UCI web repository and assign column names.
data_df = pd.read_csv(dataset_path)
data_df.head()

,STATEID,DISTID,PSUID,SCHOOLID,SQGOVT,SS1,SS3,SS4,SS5,SS6,SS7,SS8,SS9,SS10,SS11,SS12,SS13
0,1,2,1,1,1,1,1,1,5,1,1,25,15,1,2,3,2
1,1,2,1,1,1,2,2,3,5,1,2,26,15,1,2,3,2
2,1,2,1,1,1,3,3,3,99,1,2,35,0,,2,5,0
3,1,2,1,2,2,1,1,3,2,2,2,25,15,0,2,3,4
4,1,2,1,2,2,2,2,1,2,1,2,21,15,0,2,3,1


In [5]:
data_df = data_df.dropna()

In [6]:
# List of column names in the dataset.
cols = data_df.columns

In [7]:
data_df.size

543898

In [8]:
for col in data_df.select_dtypes(include=[object]):

    data_df['org_'+col] =  data_df[col]
    data_df[col] = data_df[col].astype('category').cat.codes
    data_df[col] = data_df[col].astype(float)
    print(col)
    print(dict(zip( data_df['org_'+ col], data_df[col] ) ))


data_df = data_df[cols]
data_df.head(10)

SS1
{'1': 1.0, '2': 12.0, '3': 23.0, '4': 34.0, '5': 42.0, '6': 43.0, '7': 44.0, '8': 45.0, '9': 46.0, '10': 2.0, '11': 3.0, '12': 4.0, '13': 5.0, '14': 6.0, '15': 7.0, '16': 8.0, '17': 9.0, '18': 10.0, '19': 11.0, '20': 13.0, '21': 14.0, '22': 15.0, '23': 16.0, '24': 17.0, '25': 18.0, '26': 19.0, '27': 20.0, '28': 21.0, '29': 22.0, '30': 24.0, '31': 25.0, '32': 26.0, '33': 27.0, '34': 28.0, '35': 29.0, '36': 30.0, '37': 31.0, '38': 32.0, '39': 33.0, '40': 35.0, '41': 36.0, '42': 37.0, '43': 38.0, '44': 39.0, '45': 40.0, '46': 41.0, ' ': 0.0}
SS3
{'1': 1.0, '2': 2.0, '3': 3.0, '5': 5.0, '4': 4.0, ' ': 0.0}
SS4
{'1': 1.0, '3': 3.0, '2': 2.0, ' ': 0.0}
SS5
{'5': 9.0, '99': 15.0, '2': 5.0, '4': 8.0, '3': 7.0, '8': 13.0, '6': 11.0, '1': 1.0, '10': 2.0, '55': 10.0, '12': 4.0, '9': 14.0, '7': 12.0, ' ': 0.0, '11': 3.0, '22': 6.0}
SS6
{'1': 1.0, '2': 2.0, ' ': 0.0, '3': 3.0}
SS7
{'1': 1.0, '2': 2.0, ' ': 0.0}
SS8
{'25': 12.0, '26': 13.0, '35': 22.0, '21': 8.0, '32': 19.0, '27': 14.0, '24': 11

,STATEID,DISTID,PSUID,SCHOOLID,SQGOVT,SS1,SS3,SS4,SS5,SS6,SS7,SS8,SS9,SS10,SS11,SS12,SS13
0,1,2,1,1,1,1.0,1.0,1.0,9.0,1.0,1.0,12.0,4.0,2.0,2.0,3.0,13.0
1,1,2,1,1,1,12.0,2.0,3.0,9.0,1.0,2.0,13.0,4.0,2.0,2.0,3.0,13.0
2,1,2,1,1,1,23.0,3.0,3.0,15.0,1.0,2.0,22.0,1.0,0.0,2.0,5.0,1.0
3,1,2,1,2,2,1.0,1.0,3.0,5.0,2.0,2.0,12.0,4.0,1.0,2.0,3.0,35.0
4,1,2,1,2,2,12.0,2.0,1.0,5.0,1.0,2.0,8.0,4.0,1.0,2.0,3.0,2.0
5,1,2,2,1,1,1.0,1.0,1.0,9.0,1.0,1.0,22.0,5.0,2.0,2.0,2.0,43.0
6,1,2,2,1,1,12.0,2.0,2.0,9.0,1.0,2.0,19.0,5.0,2.0,2.0,2.0,2.0
7,1,2,2,2,2,1.0,1.0,1.0,9.0,1.0,1.0,14.0,4.0,2.0,2.0,2.0,1.0
8,1,2,2,2,2,12.0,2.0,3.0,9.0,1.0,1.0,11.0,4.0,2.0,2.0,2.0,1.0
9,1,2,2,2,2,23.0,2.0,3.0,9.0,1.0,1.0,21.0,3.0,1.0,2.0,2.0,1.0


In [9]:
data_df['SS9'] = data_df['SS9'].astype(int)
data_df['SS9'] = np.where((data_df['SS9'] != 5), 0, 1)
data_df['SS9'].unique()


array([0, 1])

In [10]:
labels_df = data_df['SS9'].astype(int)

In [11]:
labels_df

0        0
1        0
2        0
3        0
4        0
        ..
31989    0
31990    0
31991    0
31992    0
31993    0
Name: SS9, Length: 31994, dtype: int64

In [12]:
cols = data_df.columns

In [13]:
data_df.size

543898

In [14]:
feature_names = data_df.columns
for feature_name in feature_names:  
    # Which rows have missing values?
    missing_rows = data_df[feature_name].isna()
    if missing_rows.any():  # Check if at least one row has a missing value.
        data_df[feature_name].fillna(0.0, inplace=True)  # Fill NaN with 0.
        missing_rows.rename(feature_name + "_is_missing", inplace=True)
        data_df = data_df.join(missing_rows)  # Append "is_missing" featur

In [15]:
for col in data_df.select_dtypes(include=[object]):

    data_df['org_'+col] =  data_df[col]
    data_df[col] = data_df[col].astype('category').cat.codes
    data_df[col] = data_df[col].astype(float)
    print(col)
    print(dict(zip( data_df['org_'+ col], data_df[col] ) ))


data_df = data_df[cols]
data_df.head(10)

,STATEID,DISTID,PSUID,SCHOOLID,SQGOVT,SS1,SS3,SS4,SS5,SS6,SS7,SS8,SS9,SS10,SS11,SS12,SS13
0,1,2,1,1,1,1.0,1.0,1.0,9.0,1.0,1.0,12.0,0,2.0,2.0,3.0,13.0
1,1,2,1,1,1,12.0,2.0,3.0,9.0,1.0,2.0,13.0,0,2.0,2.0,3.0,13.0
2,1,2,1,1,1,23.0,3.0,3.0,15.0,1.0,2.0,22.0,0,0.0,2.0,5.0,1.0
3,1,2,1,2,2,1.0,1.0,3.0,5.0,2.0,2.0,12.0,0,1.0,2.0,3.0,35.0
4,1,2,1,2,2,12.0,2.0,1.0,5.0,1.0,2.0,8.0,0,1.0,2.0,3.0,2.0
5,1,2,2,1,1,1.0,1.0,1.0,9.0,1.0,1.0,22.0,1,2.0,2.0,2.0,43.0
6,1,2,2,1,1,12.0,2.0,2.0,9.0,1.0,2.0,19.0,1,2.0,2.0,2.0,2.0
7,1,2,2,2,2,1.0,1.0,1.0,9.0,1.0,1.0,14.0,0,2.0,2.0,2.0,1.0
8,1,2,2,2,2,12.0,2.0,3.0,9.0,1.0,1.0,11.0,0,2.0,2.0,2.0,1.0
9,1,2,2,2,2,23.0,2.0,3.0,9.0,1.0,1.0,21.0,0,1.0,2.0,2.0,1.0


In [28]:
# Set random seed so that the results are reproducible.
np.random.seed(123456)

# Train and test indices.
train_indices, test_indices = model_selection.train_test_split(
    np.arange(data_df.shape[0]), test_size=1./3.)

# Train and test data.
x_train_df = data_df.loc[train_indices].astype(np.float32)
y_train_df = labels_df.loc[train_indices].astype(np.float32)
x_test_df = data_df.loc[test_indices].astype(np.float32)
y_test_df = labels_df.loc[test_indices].astype(np.float32)

# Convert data frames to NumPy arrays.
x_train = x_train_df.values
y_train = y_train_df.values
x_test = x_test_df.values
y_test = y_test_df.values

We're almost ready to construct and train our model, but first we'll create a couple of functions to measure performance. We're interested in two quantities: the average hinge loss (which we seek to minimize), and the recall (which we constrain).

In [29]:
np.unique(y_train)

array([0., 1.], dtype=float32)

In [30]:
np.unique(y_test)

array([0., 1.], dtype=float32)

In [31]:
from sklearn.metrics import confusion_matrix


def average_hinge_loss(labels, predictions):
  # Recall that the labels are binary (0 or 1).
    signed_labels = (labels * 2) - 1
    return np.mean(np.maximum(0.0, 1.0 - signed_labels * predictions))

def constrained_recall(labels, predictions, pred_labels): #tp/tp+fn
  # Recall that the labels are binary (0 or 1).
    cm = confusion_matrix(labels, pred_labels)
    positive_count = np.sum(labels)
    negative_count = len(labels) - np.sum(labels)
    true_positives = labels * (predictions > 0)
    true_positive_count = np.sum(true_positives)
    
  #compute tp, tp_and_fn and tp_and_fp w.r.t all classes
    tp_and_fn = cm.sum(1)
    tp = cm.diagonal()
    fp = np.sum(cm, axis=0) - tp
    tp_and_fp  = fp + tp

    precision = tp / tp_and_fp
    recall = tp / tp_and_fn
    
    #print(precision)
    #print(recall)
    
    return recall  #true_positive_count / negative_count


def constrained_precision(labels, predictions, pred_labels): #tp/tp+fp
  # Precision that the labels are binary (0 or 1).
    cm = confusion_matrix(labels, pred_labels)
    positive_count = np.sum(labels)
    tp = cm.diagonal()
    true_positives = labels * (predictions > 0)
    
    fp = np.sum(cm, axis=0) - tp
    tp_and_fp  = fp + tp
    
    true_positive_count = np.sum(true_positives)

   #compute tp, tp_and_fn and tp_and_fp w.r.t all classes
    tp_and_fn = cm.sum(1)
    

    precision = tp / tp_and_fp
    recall = tp / tp_and_fn
    
    #print(precision, tp, tp_and_fp)
    #print(recall)

    return precision  #true_positive_count / positive_count

## Constructing and Optimizing the Model

The first step is to create the [KerasPlaceholder](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/rates/keras.py)s that we'll need. Even in eager mode, these objects act similarly to graph-mode placeholders, in that they initially contain no values, but will be filled-in later (when the Keras loss function is called).

They're parameterized by a function that takes the same parameters as a Keras loss function (prediction and labels), and returns the Tensor that the placeholder should represent. In this case, tfco_predictions returns the predictions themselves, and tfco_labels returns the labels themselves, but in more complex settings, one might need to extract multiple different quantities (e.g. protected class information, the predictions of a baseline model, etc.) from the labels.

In [32]:
tfco_predictions = tfco.KerasPlaceholder(lambda _, y_pred: y_pred)
tfco_labels = tfco.KerasPlaceholder(lambda y_true, _: y_true)

The main motivation of TFCO is to make it easy to create and optimize constrained problems written in terms of linear combinations of *rates*, where a "rate" is the proportion of training examples on which an event occurs (e.g. the false positive rate, which is the number of negatively-labeled examples on which the model makes a positive prediction, divided by the number of negatively-labeled examples). Our current example (minimizing a hinge relaxation of the error rate subject to a recall constraint) is such a problem.

Using the placeholders defined above, we are now able to define the problem to optimize. The [KerasLayer](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/rates/keras.py) interface is similar to the [RateMinimizationProblem](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/rates/rate_minimization_problem.py) interface, in that its two main parameters are the expression to minimize, and a list of constraints. Unlike a [RateMinimizationProblem](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/rates/rate_minimization_problem.py), however, it also requires a list of all placeholders that are required by its inputs.

# Constrained Recall

In [33]:
context = tfco.rate_context(predictions=tfco_predictions, labels=tfco_labels)
tfco_layer_recall = tfco.KerasLayer(
    tfco.error_rate(context), [tfco.recall(context) >= recall_lower_bound],
    placeholders=[tfco_predictions, tfco_labels])

A [KerasLayer](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/tensorflow_constrained_optimization/python/rates/keras.py) plays two roles.


1.   It defines the optimization problem, in terms of an objective and constraints. To this end, it also contains the loss function that should be passed to Keras' Model.compile() method.
2.   It also contains the internal state needed by TFCO. For this reason, it must be included somewhere in the Keras model. It doesn't matter *where* it's included, since from the perspective of the model, it's an identity function. However, it must be included *somewhere*, so that the internal TFCO state will be updated during optimization.

We now construct our model. As in [README.md](https://github.com/google-research/tensorflow_constrained_optimization/tree/master/README.md), we're using a linear model with a bias. Notice that we include tfco_layer in the Sequential model, which ensures that the TFCO internal state will be updated during optimization. We also pass tfco_layer.loss to the Model.compile() function, which causes us to optimize the correct constrained objective. The placeholders that we constructed earlier will be filled-in when tfco_layer.loss() is called.

In [34]:
# You can put the tfco.KerasLayer anywhere in the sequence--its only purpose is
# to contain the slack variables, denominators, Lagrange multipliers, and loss.
# It's a NO-OP (more accurately, an identity function) as far as the model is
# concerned.
layers = []
layers.append(tf.keras.Input(shape=(x_train.shape[-1],)))
layers.append(tf.keras.layers.Dense(32, activation='relu')) 
layers.append(tf.keras.layers.Dense(32, activation='relu')) 
layers.append(tf.keras.layers.Dense(1))
layers.append(tfco_layer_recall)
model = tf.keras.Sequential(layers)

# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(1, activation=None, input_shape=(dimension,)),
#     tfco_layer
# ])
model.summary()

# Notice that we take the loss function from the tfco.KerasLayer, instead of
# using tf.keras.losses.Hinge(), as we did above.
model.compile(
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1),
    loss=tfco_layer_recall.loss)

model.fit(x=x_train, y=y_train, batch_size=128, epochs=100)
labels = y_test

y_pred = model.predict(x_test)
pred_labels = np.argmax(y_pred, axis=1)
print(pred_labels, y_test)



precisions, recall, f1_score, true_sum = metrics.precision_recall_fscore_support(y_test, pred_labels)

print("Precision =", precisions)
print("Recall=", recall)
print("F1 Score =", f1_score)
accuracy_score = metrics.accuracy_score(y_test, pred_labels)
print('Accuracy' + str(accuracy_score))


trained_predictions = np.ndarray.flatten(model.predict(x_test))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 32)                576       
_________________________________________________________________
dense_7 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 33        
_________________________________________________________________
keras_layer_1 (KerasLayer)   (None, 1)                 6         
Total params: 1,671
Trainable params: 1,666
Non-trainable params: 5
_________________________________________________________________
Epoch 1/100
167/167 [==============================] - 0s 1ms/step - loss: 1.0081
Epoch 2/100
167/167 [==============================] - 0s 914us/step - loss: 0.2303
Epoch 3/100
167/167 [==============================] - 0s 871us/step - loss: 0.0023


In [35]:
print(np.shape(labels))
np.shape(trained_predictions)

(10665,)


(10665,)

In [36]:
print("Constrained recall = ", constrained_recall(labels, trained_predictions, pred_labels))

Constrained recall =  [1. 0.]


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


In [37]:
print("Constrained average hinge loss = %f" %
      average_hinge_loss(labels, trained_predictions))


Constrained average hinge loss = 0.000367


As we hoped, the recall is extremely close to 90%&mdash;and, thanks to the fact that the optimizer uses a (hinge) proxy constraint only when needed, and the actual (zero-one) constraint whenever possible, this is the *true* recall, not a hinge approximation.

### Unconstrained Model

For comparison, let's try optimizing the same problem *without* the recall constraint:

In [38]:
layers = []
layers.append(tf.keras.Input(shape=(x_train.shape[-1],)))
layers.append(tf.keras.layers.Dense(32, activation='relu')) 
layers.append(tf.keras.layers.Dense(32, activation='relu')) 
layers.append(tf.keras.layers.Dense(1))
model = tf.keras.Sequential(layers)

model.summary()

# Notice that we take the loss function from the tfco.KerasLayer, instead of
# using tf.keras.losses.Hinge(), as we did above.
model.compile(
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=.1),
    loss=tf.keras.losses.Hinge())

model.fit(x=x_train, y=y_train, batch_size=128, epochs=100)
labels = y_test

y_pred = model.predict(x_test)
pred_labels = np.argmax(y_pred, axis=1)
print(pred_labels, y_test)



precisions, recall, f1_score, true_sum = metrics.precision_recall_fscore_support(y_test, pred_labels)

print("Precision =", precisions)
print("Recall=", recall)
print("F1 Score =", f1_score)
accuracy_score = metrics.accuracy_score(y_test, pred_labels)
print('Accuracy' + str(accuracy_score))



Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 32)                576       
_________________________________________________________________
dense_10 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
Total params: 1,665
Trainable params: 1,665
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
167/167 [==============================] - 0s 770us/step - loss: 0.5477
Epoch 2/100
167/167 [==============================] - 0s 625us/step - loss: 0.4322
Epoch 3/100
167/167 [==============================] - 0s 560us/step - loss: 0.4235
Epoch 4/100
167/167 [==============================] - 0s 534us/step - loss: 0.4165
Epoch 5/100
167/167 [=========================

167/167 [==============================] - 0s 571us/step - loss: 0.0000e+00
Epoch 86/100
167/167 [==============================] - 0s 524us/step - loss: 0.0000e+00
Epoch 87/100
167/167 [==============================] - 0s 531us/step - loss: 0.0000e+00
Epoch 88/100
167/167 [==============================] - 0s 555us/step - loss: 0.0000e+00
Epoch 89/100
167/167 [==============================] - 0s 579us/step - loss: 0.0000e+00
Epoch 90/100
167/167 [==============================] - 0s 539us/step - loss: 0.0000e+00
Epoch 91/100
167/167 [==============================] - 0s 539us/step - loss: 0.0000e+00
Epoch 92/100
167/167 [==============================] - 0s 662us/step - loss: 0.0000e+00
Epoch 93/100
167/167 [==============================] - 0s 598us/step - loss: 0.0000e+00
Epoch 94/100
167/167 [==============================] - 0s 546us/step - loss: 0.0000e+00
Epoch 95/100
167/167 [==============================] - 0s 556us/step - loss: 0.0000e+00
Epoch 96/100
167/167 [============

Because there is no constraint, the unconstrained problem does a better job of minimizing the average hinge loss. To test recall we need a more balanced data

In [39]:
trained_predictions = np.ndarray.flatten(model.predict(x_test))
print("Unconstrained average hinge loss = %f" % average_hinge_loss(
    labels, trained_predictions))
print("Unconstrained recall =", constrained_recall(labels, trained_predictions, pred_labels))
print("Unconstrained precision =", constrained_precision(labels, trained_predictions, pred_labels))


Unconstrained average hinge loss = 0.000205
Unconstrained recall = [1. 0.]
Unconstrained precision = [0.78687295        nan]


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in true_divide
